In [1]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
# from pyspark import SparkContext
# sc = SparkContext.getOrCreate()

In [2]:
import findspark
findspark.init(r'C:\Users\14126\Downloads\university_of_pittsburgh\Classes\Spring2020\2750_cloud_computing\Project2\files\spark-3.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
df_log = spark.read.csv('access_log', header=False, inferSchema=True, sep=' ')
df_log.show()

+--------------+---+---+--------------------+------+--------------------+---+-----+
|           _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6|  _c7|
+--------------+---+---+--------------------+------+--------------------+---+-----+
|10.223.157.186|  -|  -|[15/Jul/2009:14:5...|-0700]|      GET / HTTP/1.1|403|  202|
|10.223.157.186|  -|  -|[15/Jul/2009:14:5...|-0700]|GET /favicon.ico ...|404|  209|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|      GET / HTTP/1.1|200| 9157|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/js/lo...|200|10469|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/css/r...|200| 1014|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/css/9...|200| 6206|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/css/t...|200|15779|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/js/th...|200| 4492|
|10.223.157.186|  -|  -|[15/Jul/2009:15:5...|-0700]|GET /assets/js/li...|200

In [4]:
df1 = df_log.select("_c0", F.regexp_replace(F.col("_c3"), "\[", "").alias("Time"))
df1.show()

+--------------+--------------------+
|           _c0|                Time|
+--------------+--------------------+
|10.223.157.186|15/Jul/2009:14:58:59|
|10.223.157.186|15/Jul/2009:14:58:59|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:35|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:36|
|10.223.157.186|15/Jul/2009:15:50:37|
+--------------+--------------------+
only showing top 20 rows



In [5]:
df_model = df1.select("_c0", F.to_timestamp(df1.Time, 'dd/MMM/yyyy:HH:mm:ss').alias('date'))
df_model.show()

+--------------+-------------------+
|           _c0|               date|
+--------------+-------------------+
|10.223.157.186|2009-07-15 14:58:59|
|10.223.157.186|2009-07-15 14:58:59|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:35|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:36|
|10.223.157.186|2009-07-15 15:50:37|
+--------------+-------------------+
only showing top 20 rows



In [6]:
df = df_model.withColumn('year',F.to_date(df_model.date)).groupby('year').count()
df = df.withColumn("year", F.unix_timestamp("year", "dd-MM-yyyy")/10000000000)
df = df.dropna()
df.show()

+----------+-----+
|      year|count|
+----------+-----+
| 0.1258866| 2051|
|0.12814992| 7885|
|0.12497904|  115|
|0.12559248|   32|
|0.12853008| 4203|
|0.12962772| 3238|
|0.12963636| 4868|
|0.13046544| 6501|
|  0.131796| 9857|
| 0.1271304| 4564|
|0.12819312| 4848|
|0.12957588| 3536|
|0.13100112| 8722|
|0.13109616| 8993|
|0.13138992| 5632|
|0.13215924| 7051|
| 0.1279512| 7376|
|0.12923892| 4842|
| 0.1266642| 2222|
|0.12712176| 6862|
+----------+-----+
only showing top 20 rows



In [7]:
feature_columns = df.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns,outputCol="input")
data_2 = assembler.transform(df)
data_2.show()

+----------+-----+------------+
|      year|count|       input|
+----------+-----+------------+
| 0.1258866| 2051| [0.1258866]|
|0.12814992| 7885|[0.12814992]|
|0.12497904|  115|[0.12497904]|
|0.12559248|   32|[0.12559248]|
|0.12853008| 4203|[0.12853008]|
|0.12962772| 3238|[0.12962772]|
|0.12963636| 4868|[0.12963636]|
|0.13046544| 6501|[0.13046544]|
|  0.131796| 9857|  [0.131796]|
| 0.1271304| 4564| [0.1271304]|
|0.12819312| 4848|[0.12819312]|
|0.12957588| 3536|[0.12957588]|
|0.13100112| 8722|[0.13100112]|
|0.13109616| 8993|[0.13109616]|
|0.13138992| 5632|[0.13138992]|
|0.13215924| 7051|[0.13215924]|
| 0.1279512| 7376| [0.1279512]|
|0.12923892| 4842|[0.12923892]|
| 0.1266642| 2222| [0.1266642]|
|0.12712176| 6862|[0.12712176]|
+----------+-----+------------+
only showing top 20 rows



In [8]:
train, test = data_2.randomSplit([0.8, 0.2])
lin_reg = LinearRegression(featuresCol="input", labelCol="count", predictionCol='prediction')
model = lin_reg.fit(train)
eval_summary = model.evaluate(test)


In [9]:
print(model.summary.meanAbsoluteError)
print(eval_summary.rootMeanSquaredError)
print(eval_summary.r2)

1778.012258572827
1823.6775793635018
0.591566159730869


In [10]:
predictions = model.transform(test)
predictions.select(predictions.columns).show()

+----------+-----+------------+------------------+
|      year|count|       input|        prediction|
+----------+-----+------------+------------------+
|0.12853008| 4203|[0.12853008]| 5360.536755179201|
|0.12962772| 3238|[0.12962772]| 6650.771739373944|
|0.13046544| 6501|[0.13046544]|7635.4804334444925|
| 0.1271304| 4564| [0.1271304]|3715.2649878623197|
|0.12827088| 6315|[0.12827088]|  5055.85679826868|
|0.13146768| 9256|[0.13146768]| 8813.576266831922|
| 0.1310184| 5085| [0.1310184]| 8285.464341520332|
|0.12788208| 4465|[0.12788208]| 4598.836862902885|
| 0.1290402| 5638| [0.1290402]| 5960.163837043394|
|0.12703536| 1635|[0.12703536]| 3603.549003661814|
|0.13011984| 5622|[0.13011984]| 7229.240490897122|
|0.13178736|10790|[0.13178736]| 9189.348213688238|
|0.12650868| 2848|[0.12650868]| 2984.456257883867|
|0.12489264|   34|[0.12489264]| 1084.861359868024|
|0.13169232| 8007|[0.13169232]| 9077.632229487703|
|0.13021488| 7708|[0.13021488]| 7340.956475097686|
|0.12772656| 5842|[0.12772656]|

In [11]:
path = 'linear_reg_spark_part3_model'
model.write().overwrite().save(path)